# Deep Convolutional Generative Adversatial Network

##### Importing libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
from time import time
import os
os.chdir('C:/Users/Nicolas/Documents/Data/Faces')
import argparse
import math
import itertools
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch

##### Loading all file names

In [2]:
files = glob('combined/*.jpg')

##### Keeping all correctly formatted labels

In [3]:
np.unique([i[-34] for i in files], return_counts=True)

(array(['0', '1', '9', '_'], dtype='<U1'),
 array([12387, 11316,     1,     5], dtype=int64))

##### Removing problematic target names

0 is men, 1 is women

In [4]:
faces = [i for i in files if (i[-34] in ('0', '1')) and len(i[-37:-35].strip('\\').strip('d'))  == 2 ]

In [5]:
y = [i[-34] for i in files if (i[-34] in ('0', '1')) and len(i[-37:-35].strip('\\').strip('d')) > 1 ]

In [ ]:
assert len(y) == len(faces), 'The X and Y are not of the same length!'

#### This is the shape width/height

In [ ]:
dim = 60

#### Cropping function

In [ ]:
def crop(img):
    if img.shape[0]<img.shape[1]:
        x = img.shape[0]
        y = img.shape[1]
        crop_img = img[: , int(y/2-x/2):int(y/2+x/2)]
    else:
        x = img.shape[1]
        y = img.shape[0]
        crop_img = img[int(y/2-x/2):int(y/2+x/2) , :]

    return crop_img

##### Loading and cropping images

In [ ]:
print('Scaling...', end='')
start = time()
x = []
num_to_load = len(faces)  
for ix, file in enumerate(faces[:num_to_load]): 
    image = plt.imread(file, 'jpg')
    image = Image.fromarray(image).resize((dim, dim)).convert('L')
    image = crop(np.array(image))
    x.append(image)
print(f'\rDone. {int(time() - start)} seconds')
y = y[:num_to_load]

##### Turning the pictures into arrays

In [ ]:
x = np.divide(np.array(x, dtype=np.float32), 255).reshape(-1, 1, 60, 60)
y = np.array(y, dtype=np.float32)

##### Turning the targets into a 2D matrix

In [11]:
assert x.ndim == 4, 'The input is the wrong shape!'

In [12]:
yy, xx = y.nbytes, x.nbytes

In [13]:
print(f'The size of X is {xx:,} bytes and the size of Y is {yy:,} bytes.')

The size of X is 297,187,200 bytes and the size of Y is 82,552 bytes.


In [14]:
files, faces = None, None

In [15]:
print('Scaling...', end='')
image_size = x.shape[1] * x.shape[1]
print('\rDone.     ')

Done.     


In [16]:
if torch.cuda.is_available():
    x = torch.from_numpy(x) 
    y = torch.from_numpy(y)
    print('Tensors successfully flushed to CUDA.')
else:
    print('CUDA not available!')

Tensors successfully flushed to CUDA.


In [17]:
from torchvision.transforms import Resize

##### Making a dataset class

In [18]:
class Face():
    
    def __init__(self):
        self.len = x.shape[0]
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        return x[index], y[index].unsqueeze(0) 
    
    def __len__(self):
        return self.len

##### Instantiating the class

In [19]:
data = Face()

##### Parsing the args

In [20]:
os.makedirs("images", exist_ok=True)

parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=25, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=32, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=256, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=60, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument("--sample_interval", type=int, default=40, help="interval between image sampling")
opt, unknown = parser.parse_known_args()
print(opt)
print('\n', unknown)

Namespace(b1=0.5, b2=0.999, batch_size=32, channels=1, img_size=60, latent_dim=256, lr=0.0002, n_cpu=8, n_epochs=25, sample_interval=40)

 ['-f', 'C:\\Users\\Nicolas\\AppData\\Roaming\\jupyter\\runtime\\kernel-6e27e8fb-4386-4242-8d70-50816aeabddb.json']


In [21]:
cuda = True if torch.cuda.is_available() else False

In [22]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [23]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = opt.img_size // 4 ## 15
        self.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * self.init_size ** 2)) # (100, 128*15^2) 28800
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, opt.channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

In [24]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(opt.channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = opt.img_size // (2 ** 4) # gives 3
        self.adv_layer = nn.Sequential(nn.Linear(128 * 4 ** 2, 1), nn.Sigmoid()) # s'attend à (1152, 1)

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1) # torch.Size([64, 2048])
        validity = self.adv_layer(out)

        return validity

In [25]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [26]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (adv_laye

In [27]:
# Configure data loader
dataloader = torch.utils.data.DataLoader(data,
    batch_size=opt.batch_size,
    shuffle=True,
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [28]:
for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()
        
        if not os.path.isdir('dcgan'):
            os.mkdir('dcgan')
        
        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "dcgan/%d.png" % batches_done, nrow=5, normalize=True)

    print(
        "[Epoch %d/%d] [D loss: %f] [G loss: %f]"
        % (epoch, opt.n_epochs, d_loss.item(), g_loss.item())
    )

[Epoch 0/25] [D loss: 0.691821] [G loss: 0.695691]
[Epoch 1/25] [D loss: 0.695268] [G loss: 0.696495]
[Epoch 2/25] [D loss: 0.692491] [G loss: 0.690659]
[Epoch 3/25] [D loss: 0.694450] [G loss: 0.717530]
[Epoch 4/25] [D loss: 0.690645] [G loss: 0.670912]
[Epoch 5/25] [D loss: 0.690027] [G loss: 0.696533]
[Epoch 6/25] [D loss: 0.694171] [G loss: 0.687500]
[Epoch 7/25] [D loss: 0.689855] [G loss: 0.691292]
[Epoch 8/25] [D loss: 0.695701] [G loss: 0.697978]
[Epoch 9/25] [D loss: 0.691632] [G loss: 0.693510]
[Epoch 10/25] [D loss: 0.693704] [G loss: 0.691005]
[Epoch 11/25] [D loss: 0.692534] [G loss: 0.695347]
[Epoch 12/25] [D loss: 0.691024] [G loss: 0.695503]


KeyboardInterrupt: 

In [ ]:
if not os.path.isdir('saves'):
    os.mkdir('saves')

In [ ]:
torch.save({
            'discriminator_state_dict': discriminator.state_dict(),
            'generator_state_dict': generator.state_dict(),
            'optimizer_discriminator_state_dict': optimizer_D.state_dict(),
            'generator_state_dict': optimizer_G.state_dict()
            }, r'C:\Users\Nicolas\Documents\Data\faces\saves')

In [ ]:
discriminator = Discriminator(*args, **kwargs)
generator = Generator(*args, **kwargs)
optimizer_discriminator = Discriminator(*args, **kwargs)
generator = Generator(*args, **kwargs)

checkpoint = torch.load('')
discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
generator.load_state_dict(checkpoint['generator_state_dict'])
optimizer_discriminator.load_state_dict(checkpoint['optimizer_discriminator_state_dict'])
optimizer_generator.load_state_dict(checkpoint['optimizer_generator_state_dict'])

discriminator.eval()
generator.eval()
# - or -
discriminator.train()
generator.train()